In [41]:
%%configure -f
{"executorMemory":"5G", "numExecutors":2, "executorCores":3, "conf":
{"spark.dynamicAllocation.enabled": "false"}}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1683207519086_0004,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1683207519086_0004,spark,idle,Link,Link,None,✔


In [1]:
val bucketname = "unibo-bd-antonelli2023"
val path_ml_tripdata =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-01.parquet"
//"s3a://"+bucketname+"/datasets/project/2017/"
val path_ml_tripdata802 =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-02.parquet"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683811633231_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd-antonelli2023
path_ml_tripdata: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-01.parquet
path_ml_tripdata802: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-02.parquet
res3: String = application_1683811633231_0001
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1683811633231_0001/


In [2]:
import spark.implicits._
val parquetFileDF = spark.read.parquet(path_ml_tripdata)
val parquetFileDF802 = spark.read.parquet(path_ml_tripdata802)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
parquetFileDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]
parquetFileDF802: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


In [3]:
import java.sql.Timestamp
case class TaxiTrip(
  licenseClass:String,
  license:String,  
  request:Timestamp,
  pickup:Timestamp,
  dropoff:Timestamp,
  distance:Double,
  startloc:Long,
  endloc:Long,
  //time:Long,
  fare:Double,
)

object TaxiTrip {
    def extract(row:org.apache.spark.sql.Row) = {
        val licenseClass = row.getString(0)
        val license = row.getString(1)
        val request = row.getTimestamp(3)
        val pickup = row.getTimestamp(5)
        val dropoff = row.getTimestamp(6)
        val distance = row.getDouble(9)
        val startloc = row.getLong(7)
        val endloc = row.getLong(8)
        //val time = row.getLong(10)
        val fare = row.getDouble(11)
        
        new TaxiTrip(licenseClass,license,request,pickup,dropoff,distance,startloc,endloc,fare)
        

        //new TaxiTrip(license,request,pickup,dropoff,startloc,endloc,distance,time,fare)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.sql.Timestamp
defined class TaxiTrip
defined object TaxiTrip
Companions must be defined together; you may wish to use :paste mode for this.


In [4]:
import java.util.Calendar
def getDayTime(time: Long): Calendar = {
    var date:Calendar = Calendar.getInstance();
    date.setTimeInMillis(time);
    date
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
getDayTime: (time: Long)java.util.Calendar


In [5]:
import org.apache.spark.HashPartitioner
val p = new HashPartitioner(12)
val data_rdd = parquetFileDF.rdd
val data_rdd802 = parquetFileDF802.rdd
val rddTaxiTrip = data_rdd.map(TaxiTrip.extract)
val rddTaxiTrip2 = data_rdd802.map(TaxiTrip.extract)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.HashPartitioner
p: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@c
data_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[9] at rdd at <console>:29
data_rdd802: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[15] at rdd at <console>:29
rddTaxiTrip: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[16] at map at <console>:30
rddTaxiTrip2: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[17] at map at <console>:30


In [6]:
rddTaxiTrip.map(x => x.startloc).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: Long = 11908468


In [7]:
//prova 1
rddTaxiTrip.map(x => x.startloc).take(5)
rddTaxiTrip.map(x => x.license).take(5)
rddTaxiTrip.map(x => x.request).take(5)
rddTaxiTrip.map(x => x.pickup).take(5)
rddTaxiTrip.map(x => x.request.getTime()).take(5)
rddTaxiTrip.map(x => x.fare).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res9: Array[Long] = Array(230, 152, 233, 142, 143)
res10: Array[String] = Array(B02682, B02682, B02764, B02764, B02764)
res11: Array[java.sql.Timestamp] = Array(2021-01-01 00:28:09.0, 2021-01-01 00:45:56.0, 2021-01-01 00:21:15.0, 2021-01-01 00:39:12.0, 2021-01-01 00:46:11.0)
res12: Array[java.sql.Timestamp] = Array(2021-01-01 00:33:44.0, 2021-01-01 00:55:19.0, 2021-01-01 00:23:56.0, 2021-01-01 00:42:51.0, 2021-01-01 00:48:14.0)
res13: Array[Long] = Array(1609460889000, 1609461956000, 1609460475000, 1609461552000, 1609461971000)
res14: Array[Double] = Array(22.28, 18.36, 14.05, 7.91, 27.11)


In [8]:
//prova 2
rddTaxiTrip.map(x => x.request.getTime()).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res16: Array[Long] = Array(1609460889000, 1609461956000, 1609460475000, 1609461552000, 1609461971000)


In [9]:
//prova 3
rddTaxiTrip.map(x => x.licenseClass).take(5)
rddTaxiTrip.map(x => x.license).take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res18: Array[String] = Array(HV0003, HV0003, HV0003, HV0003, HV0003)
res19: Array[String] = Array(B02682, B02682, B02764, B02764, B02764)


In [10]:
rddTaxiTrip.map(x => x.startloc).distinct().count()
rddTaxiTrip.map(x => x.license).distinct().count()
rddTaxiTrip2.map(x => x.startloc).distinct().count()
rddTaxiTrip2.map(x => x.license).distinct().count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res20: Long = 262
res21: Long = 32
res22: Long = 261
res23: Long = 32


VIAGGI PIU LUNGHI IN TERMINE DI KM, IN RELAZIONE CON LA DURATA DEL VIAGGIO

In [11]:
val query1 = rddTaxiTrip.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query1802 = rddTaxiTrip2.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[44] at map at <console>:29
query1802: org.apache.spark.rdd.RDD[(Double, Long)] = MapPartitionsRDD[45] at map at <console>:29


In [12]:
val query1union = query1.union(query1802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query1union: org.apache.spark.rdd.RDD[(Double, Long)] = UnionRDD[46] at union at <console>:30


In [13]:
query1union.sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res24: Array[(Double, Long)] = Array((738.95,666), (527.11,575), (512.5,541), (480.73,512), (454.49,452), (432.359,379), (417.77,469), (408.19,440), (389.65,370), (381.95,444))


VIAGGI PIU COSTOSI, IN RELAZIONE CON LA SOMMA TRA LUNGHEZZA E LA DURATA DEL VIAGGIO

In [14]:
val query2 = rddTaxiTrip.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))
val query2802 = rddTaxiTrip2.filter(_.distance > 10).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000) + x.distance)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[54] at map at <console>:29
query2802: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[56] at map at <console>:29


In [15]:
val query2union = query2.union(query2802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query2union: org.apache.spark.rdd.RDD[(Double, Double)] = UnionRDD[57] at union at <console>:30


In [16]:
query2union.sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res25: Array[(Double, Double)] = Array((1532.67,1404.95), (1374.73,906.49), (1361.13,759.65), (1308.56,886.77), (1215.83,825.95), (1159.66,626.4), (1149.87,992.73), (1149.03,452.39), (1082.69,536.21), (1073.88,629.78))


MEDIA DEI TEMPI DI ATTESA TRA RICHIESTA E PARTENZA, IN BASE ALLA ZONA DI PARTENZA

In [17]:
val query3 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).partitionBy(p)
val query3802 = rddTaxiTrip2.map(x => (x.startloc, (x.pickup.getTime() - x.request.getTime())/(60*1000))).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query3: org.apache.spark.rdd.RDD[(Long, Long)] = ShuffledRDD[65] at partitionBy at <console>:30
query3802: org.apache.spark.rdd.RDD[(Long, Long)] = ShuffledRDD[67] at partitionBy at <console>:30


In [18]:
//query2.collect()
val query3union = query3.union(query3802).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query3union: org.apache.spark.rdd.RDD[(Long, Long)] = PartitionerAwareUnionRDD[68] at union at <console>:31


In [19]:
//query3union.groupByKey().map({case(k,v) => (v.sum/v.size,k)}).sortByKey(false).take(2)
query3union.groupByKey().map({case(k,v) => (v,k)}).sortByKey(false).take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 38.0 failed 4 times, most recent failure: Lost task 1.3 in stage 38.0 (TID 122) (ip-172-31-17-117.ec2.internal executor 1): java.lang.NullPointerException

Driver stacktrace:
  at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2863)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2799)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2798)
  at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
  at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2798)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1239)
  at org.apache

In [20]:
//CAPIRE QUALE è IL RISULTATO
val query5 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query51 = rddTaxiTrip2.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query5union = query5.union(query51).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[75] at partitionBy at <console>:32
query51: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[78] at partitionBy at <console>:31
query5union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[79] at union at <console>:30


In [21]:
query5union.map({case(k,v) => (k,v._1/v._2)}).map({case(k,v) => (v,k)}).sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res30: Array[(Double, Long)] = Array((37.86046511627907,1), (35.26170798898072,1), (30.010265920054298,132), (29.887379973950182,132), (29.043478260869566,2), (26.88235294117647,110), (25.642857142857142,110), (25.2119881087581,138), (23.692460239286607,138), (22.56801909307876,27))


IN MEDIA QUANTI VIAGGI FA CIASCUNA LICENZA

In [22]:
val q = rddTaxiTrip.map(x => (x.license,1)).
reduceByKey(_+_).
aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Media: " + (q._1/q._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

q: (Int, Int) = (11908468,32)
res31: String = Media: 372139


CLASSE DI LICENZA CHE FA PIU VIAGGI, CON LA PRESENZA ANCHE DELLA TARIFFA CLIENTE

In [23]:
rddTaxiTrip.map(x => (x.licenseClass,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(HV0003,1.468278860287933E8,8704128)
(HV0005,5.509768345985989E7,3094325)
(HV0004,2816007.860000914,110015)


LICENZA CHE FA PIU VIAGGI, CON LA PRESENZA ANCHE DELLA TARIFFA CLIENTE

In [24]:
rddTaxiTrip.map(x => (x.license,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(B02510,5.503748608986006E7,3091000)
(B02764,1.709378720999917E7,1009388)
(B02872,1.5509604710001156E7,924960)
(B02875,1.2014741050004447E7,735450)
(B02765,1.0189792220006326E7,591242)
(B02869,7744772.440006703,452098)
(B02887,5608155.9400040135,333768)
(B02871,5581241.000004057,330085)
(B02682,5456731.730003582,321599)
(B02866,5284853.590003659,309274)
(B02864,5203100.810003353,316395)
(B02878,5125333.480003258,312013)
(B02617,4884453.930003084,281432)
(B02883,4582633.870002626,268391)
(B02884,4354322.950002186,257674)
(B02882,4097472.360001952,241988)
(B02876,3929723.280001779,230732)
(B02867,3730036.2400015946,217449)
(B02879,3572925.7100014733,216993)
(B02877,3490842.3500013095,208986)
(B02835,3355979.0300012426,200129)
(B02888,3032522.3300008504,177542)
(B02800,2776613.2300008354,108146)
(B02889,2500286.6500002183,149398)
(B02836,2385524.3700000723,140365)
(B02395,2056890.0899998054,124107)
(B02880,2039189.9299998167,119173)
(B02870,1768859.4399998628,106975)
(B02865,1385092.24999

MAP PER QUERY GROUP BY LOCATION/TIME, REDUCE BY KEY PER CONTARE LE OCCORRENZE DEI RECORD, 
MAP PER SCAMBIARE CHIAVE-VALORE PER POTER ORDINARE I VALORI PER VEDERE LE ZONE/PERIODI PIU GETTONATI


In [25]:
rddTaxiTrip.map(x => (x.startloc, 1)).reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

val query4 = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)
val query4802 = rddTaxiTrip2.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.HOUR_OF_DAY), 1)).partitionBy(p)

val query4union = query4.union(query4802).cache()

query4union.reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res34: Array[(Int, Long)] = Array((219931,61), (178570,76), (155806,42), (149385,37), (128972,17), (128402,244), (127347,79), (127190,225), (125635,39), (122920,89), (122537,188), (118566,74), (116312,35), (108060,7), (102600,231), (101852,75), (99197,181), (99041,132), (97275,168), (96243,41), (93193,170), (93155,234), (93149,72), (92634,36), (92091,129), (90715,82), (90197,48), (88429,69), (87167,249), (86414,169), (85722,213), (84951,68), (84866,236), (83573,256), (83416,243), (83014,255), (82550,18), (82485,97), (81148,247), (81059,237), (80466,80), (80002,91), (79979,130), (79782,235), (79588,254), (79100,198), (78961,159), (78809,112), (78545,246), (76635,26), (76143,138), (76035,107), (75845,177), (75239,140), (75048,141), (74814,137), (74173,78), (74147,226), (73517,161), (72586...
query4: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[110] at partitionBy at <console>:32
query4802: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[112] at partitionBy at <console>:31
query4

In [26]:
val tableau = rddTaxiTrip.map(x => (getDayTime(x.pickup.getTime()).get(Calendar.DAY_OF_WEEK), 1)).
                    reduceByKey((x,y) => x + y).map({case(k,v) => (v,k)}).sortByKey(false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tableau: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[124] at sortByKey at <console>:31


DISTANZA MEDIA PERCORSA DAI TAXI IN UN SINGOLO VIAGGIO, IN RELAZIONE ALLA ZONA --> vale come query finale

In [27]:
val query5 = rddTaxiTrip.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)
val query51 = rddTaxiTrip2.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).partitionBy(p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[128] at partitionBy at <console>:31
query51: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[132] at partitionBy at <console>:31


In [28]:
val query5union = query5.union(query51).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query5union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[133] at union at <console>:30


In [29]:
query5union.map({case(k,v) => (k,v._1/v._2)}).map({case(k,v) => (v,k)}).sortByKey(false).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res39: Array[(Double, Long)] = Array((20.136333333333326,1), (19.978372093023246,1), (15.583862644195618,132), (14.864652072083206,132), (11.889173913043477,2), (10.915017273061434,138), (10.765875894988065,27), (10.74933302682418,138), (10.515,199), (10.507823529411766,110), (10.40900593471811,27), (9.162642857142858,110), (9.14,199), (8.924624093697727,46), (8.61814971482889,46), (7.930163748712669,44), (7.793376006573554,201), (7.706209863588672,265), (7.667520547945171,201), (7.6189385377943255,44), (7.448386711684551,101), (7.406741350906092,204), (7.399251434034412,194), (7.375441467436126,204), (7.307814302191469,265), (7.277923984132074,101), (7.179391827697307,117), (7.034868680734756,15), (6.987986581402682,19), (6.9764677950164495,194), (6.974829246935218,117), (6.96867785358...


SQL

In [31]:
parquetFileDF.createOrReplaceTempView("parquetFile")
val parkDF = spark.sql("Select * from parquetFile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

parkDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


In [32]:
parkDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res41: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]
